In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Ai_Project/Pdf_Extraction/yolov5/
!pip install -r requirements.txt

In [ ]:
%cd /content/drive/MyDrive/Ai_Project/Pdf_Extraction/yolov5/
!python train.py --data coco.yaml --epochs 300 --weights 'yolov5x.pt'  --batch-size 4

In [ ]:
%cd /content/drive/MyDrive/Ai_Project/Pdf_Extraction/yolov5/
!python detect.py --weights /content/drive/MyDrive/Ai_Project/Pdf_Extraction/yolov5/runs/train/exp3_5X/weights/best.pt --source /content/drive/MyDrive/Ai_Project/Pdf_Extraction/yolov5/val/

In [ ]:
!pip install paddleocr --upgrade -q

In [ ]:
!pip install paddlepaddle paddleocr -q

In [ ]:
import torch
weights_path = "/content/drive/MyDrive/Ai_Project/Pdf_Extraction/yolov5/runs/train/exp3_5X/weights/best.pt"
model = torch.hub.load('/content/drive/MyDrive/Ai_Project/Pdf_Extraction/yolov5','custom', path=weights_path,force_reload=True,source='local')
model.conf = 0.5 

In [ ]:
def get_crop(numpy_image,xmin,ymin,xmax,ymax):
  crop_image = numpy_image[ymin:ymax, xmin:xmax] 
  return crop_image

In [ ]:
from glob import glob
import cv2
from paddleocr import PaddleOCR, draw_ocr
import os

ocr = PaddleOCR(use_angle_cls=True, lang='en')  # need to run only once to download and load model into memory


def get_ocr_data(frame):
    try:
        result = ocr.ocr(frame, cls=True)
        result = result[0]
        boxes = [line[0] for line in result]
        txts = [line[1][0] for line in result]
        scores = [line[1][1] for line in result]
        # ocr_image = draw_ocr(frame, boxes, txts, scores, font_path='./simfang.ttf')

        return txts, frame

    except:

        return [], frame

In [ ]:
import cv2
import glob
import pandas as pd
files = glob.glob("/content/drive/MyDrive/Ai_Project/Pdf_Extraction/yolov5/val/*.png")

response = []

response_dict = {}

for file in files:
  image_path = file
  img = cv2.imread(image_path)
  results = model(img)
  print(results)
  results_dict = results.pandas().xyxy[0].to_dict()
  labels = results.pandas().xyxy[0]
  labels = list(labels['name'])
 
  filter_dict = {}
  filter_dict["file"] = file

  for i in range(len(labels)):
    xmin = int(list(results_dict['xmin'].values())[i])
    ymin = int(list(results_dict['ymin'].values())[i])
    xmax = int(list(results_dict['xmax'].values())[i])
    ymax = int(list(results_dict['ymax'].values())[i])
    label = list(results_dict['name'].values())[i]

    crop_image = get_crop(img,xmin,ymin,xmax,ymax)
   
    ocr_data, ocr_frame = get_ocr_data(crop_image)

    try:
      filter_dict[label] = ocr_data[1]
    except:
      filter_dict[label] = ""
    print("ocr_data \n\n ",ocr_data)

  print("filter_dict \n\n ",filter_dict)
  response.append(filter_dict)

df = pd.DataFrame(response)

print(df)
df.to_json('results.json')
df.to_csv("results.csv")